<a href="https://colab.research.google.com/github/Vikram-Ahuja/Stock_Portfolios/blob/main/Shorted_Stocks_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests as rq
from math import floor,isnan
import alpaca_trade_api as trade_api

In [ ]:
def price_scrape(stock):
    url = f"https://in.finance.yahoo.com/quote/{stock}"

    data = rq.get(url).text
    soup = BeautifulSoup(data,"lxml")
    price2 = soup.find_all("span")
    price = price2[19].text
    price = price.replace(",","")
    price = price.replace("$" ,"")
    return float (price)

In [ ]:
rejected_stocks2=[]

for stock in rejected_stocks:
    stock = stock.replace(".","-")
    print(stock)
    url =f"https://in.finance.yahoo.com/quote/{stock}/key-statistics"
    data = pd.read_html(url)
    short = data[2].iloc[8,1]
    short = short.replace("%","")
    short = short.replace(",","")

    short_frame = short_frame.append(
        pd.Series([
            stock,
            float (short)
        ],
        index = col),
        ignore_index= True
    )


In [ ]:
stocks = pd.read_csv("/content/drive/MyDrive/S&P_500.csv")

col = ["Symbol" , "Short_Of_Float_Percent"]

short_frame = pd.DataFrame(columns=col)

rejected_stocks=[]

for stock in stocks.Symbol:
    stock = stock.replace(".","-")
    try:
        url =f"https://in.finance.yahoo.com/quote/{stock}/key-statistics"
        data = pd.read_html(url)

        short = data[2].iloc[8,1]
        short = short.replace("%","")
        short = short.replace(",","")

        short_frame = short_frame.append(
            pd.Series([
                       stock,
                       float (short)
            ],
            index = col),
            ignore_index= True
        )
    except:
        print(stock)
        rejected_stocks.append(stock)




In [ ]:
short_frame = short_frame.sort_values("Short_Of_Float_Percent",ascending=False)
short_frame = short_frame.reset_index(drop=True)

short_frame = short_frame[:50]

In [ ]:
val = []

for stock in short_frame.Symbol:
    val.append(price_scrape(stock))

short_frame.insert(2,"Price",val)


In [ ]:
short_frame = short_frame.drop("shares_to_buy",axis=1)

In [ ]:
shares_to_buy = []
portfolio_size = float(input("SIZE OF YOUR PORTFOLIO : "))
position_size = portfolio_size/len(short_frame)
for i in range(len(short_frame)):
    shares_to_buy.append(int(floor(position_size/short_frame.Price[i])))

short_frame.insert(3,"shares_to_buy",shares_to_buy)


In [ ]:
api_key = "PKAMEQNMZCA2AHDJ3ODL"
api_secret_key="8K2sB0sjvJOpgL0lRi0FQFpM9SXRLsOqPlRkH6Qg"
api_base_url="https://paper-api.alpaca.markets"
api = trade_api.REST(key_id=api_key, secret_key=api_secret_key, base_url=api_base_url)

print (api.get_account().status)

for i in range (len(short_frame)):
    volume = int (short_frame.shares_to_buy[i])
    sym = short_frame.Symbol[i]
    print(volume,sym)
    api.submit_order(sym,volume,"buy","market","gtc")
    